<a href="https://colab.research.google.com/github/Vietanh239/iwslt15/blob/main/Problem3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn.functional as F
import torchtext

import requests
import tarfile

In [2]:
MODELNAME1 = 'iwslt15-en-vi-rnn.model'
MODELNAME2 = 'iwslt15-en-vi-lstm.model'
EPOCH = 10
BATCHSIZE = 16
LR = 0.0001
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# PREPROCESSING

## Import Data

In [3]:
url = 'https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/'
train_en = [line.split() for line in requests.get(url+'train.en').text.splitlines()]
train_vi = [line.split() for line in requests.get(url+'train.vi').text.splitlines()]
test_en = [line.split() for line in requests.get(url+'tst2013.en').text.splitlines()]
test_vi = [line.split() for line in requests.get(url+'tst2013.vi').text.splitlines()]

In [4]:
for i in range(10):
  print(train_en[i])
  print(train_vi[i])
print('# of line', len(train_en), len(train_vi), len(test_en), len(test_vi))

['Rachel', 'Pike', ':', 'The', 'science', 'behind', 'a', 'climate', 'headline']
['Khoa', 'học', 'đằng', 'sau', 'một', 'tiêu', 'đề', 'về', 'khí', 'hậu']
['In', '4', 'minutes', ',', 'atmospheric', 'chemist', 'Rachel', 'Pike', 'provides', 'a', 'glimpse', 'of', 'the', 'massive', 'scientific', 'effort', 'behind', 'the', 'bold', 'headlines', 'on', 'climate', 'change', ',', 'with', 'her', 'team', '--', 'one', 'of', 'thousands', 'who', 'contributed', '--', 'taking', 'a', 'risky', 'flight', 'over', 'the', 'rainforest', 'in', 'pursuit', 'of', 'data', 'on', 'a', 'key', 'molecule', '.']
['Trong', '4', 'phút', ',', 'chuyên', 'gia', 'hoá', 'học', 'khí', 'quyển', 'Rachel', 'Pike', 'giới', 'thiệu', 'sơ', 'lược', 'về', 'những', 'nỗ', 'lực', 'khoa', 'học', 'miệt', 'mài', 'đằng', 'sau', 'những', 'tiêu', 'đề', 'táo', 'bạo', 'về', 'biến', 'đổi', 'khí', 'hậu', ',', 'cùng', 'với', 'đoàn', 'nghiên', 'cứu', 'của', 'mình', '--', 'hàng', 'ngàn', 'người', 'đã', 'cống', 'hiến', 'cho', 'dự', 'án', 'này', '--', 'một

## Make Vocab

In [5]:
def make_vocab(train_data, min_freq):
  vocab = {}
  for tokenlist in train_data:
    for token in tokenlist:
      if token not in vocab:
        vocab[token] = 0
      vocab[token] += 1 # Đếm các từ data 
  vocablist = [('<unk>',0),('<pad>', 0), ('<cls>', 0), ('<eos>', 0)] # Danh sách các tuple đặc trưng trước
  vocabidx = {} # dict của tuple
  for token, freq in vocab.items(): # Từ đó , với số lượng của từ đó trong train data 
    if freq >= min_freq: # Số lượng từ mà > 3
      idx = len(vocablist) # đang là 4 
      vocablist.append((token, freq)) #Từ đó , với số lượng của từ đó trong train data 
      vocabidx[token] = idx  # Từ điển (dict)  
  vocabidx['<unk>'] = 0
  vocabidx['<pad>'] = 1
  vocabidx['<cls>'] = 2
  vocabidx['<eos>'] = 3
  return vocablist, vocabidx

vocablist_en, vocabidx_en = make_vocab(train_en, 3)
vocablist_vi, vocabidx_vi = make_vocab(train_vi, 3)

print('vocab size en: ', len(vocablist_en))
print('vocab size vi: ', len(vocablist_vi))

vocab size en:  24420
vocab size vi:  10666


In [6]:
def preprocess(data, vocabidx):
  rr = []
  for tokenlist in data:
    tkl = ['<cls>']
    for token in tokenlist:
      tkl.append(token if token in vocabidx else '<unk>')
    tkl.append('<eos>')
    rr.append((tkl))
  return rr
 
train_en_prep = preprocess(train_en, vocabidx_en)
train_vi_prep = preprocess(train_vi, vocabidx_vi)
test_en_prep = preprocess(test_en, vocabidx_en)
for i in range(5):
  print(train_en_prep[i])
  print(train_vi_prep[i])
  print(test_en_prep[i])

['<cls>', 'Rachel', 'Pike', ':', 'The', 'science', 'behind', 'a', 'climate', 'headline', '<eos>']
['<cls>', 'Khoa', 'học', 'đằng', 'sau', 'một', 'tiêu', 'đề', 'về', 'khí', 'hậu', '<eos>']
['<cls>', 'When', 'I', 'was', 'little', ',', 'I', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'I', 'grew', 'up', 'singing', 'a', 'song', 'called', '&quot;', 'Nothing', 'To', '<unk>', '.', '&quot;', '<eos>']
['<cls>', 'In', '4', 'minutes', ',', 'atmospheric', 'chemist', 'Rachel', 'Pike', 'provides', 'a', 'glimpse', 'of', 'the', 'massive', 'scientific', 'effort', 'behind', 'the', 'bold', 'headlines', 'on', 'climate', 'change', ',', 'with', 'her', 'team', '--', 'one', 'of', 'thousands', 'who', 'contributed', '--', 'taking', 'a', 'risky', 'flight', 'over', 'the', 'rainforest', 'in', 'pursuit', 'of', 'data', 'on', 'a', 'key', 'molecule', '.', '<eos>']
['<cls>', 'Trong', '4', 'phút', ',', 'chuyên', 'gia', 'hoá', 'học', 'khí', 'quyển', 'Rachel', 'Pike', 'giới', 'thiệu

## Make batch & Padding

In [7]:
train_data = list(zip(train_en_prep, train_vi_prep))
train_data.sort(key = lambda x: (len(x[0]), len(x[1])))

test_data = list(zip(test_en_prep, test_en, test_vi))

for i in range(5):
  print(train_data[i])
for i in range(5):
  print(test_data[i])

(['<cls>', '<eos>'], ['<cls>', '<eos>'])
(['<cls>', '<eos>'], ['<cls>', '<eos>'])
(['<cls>', '<eos>'], ['<cls>', '<eos>'])
(['<cls>', '<eos>'], ['<cls>', '<eos>'])
(['<cls>', '<eos>'], ['<cls>', '<eos>'])
(['<cls>', 'When', 'I', 'was', 'little', ',', 'I', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'I', 'grew', 'up', 'singing', 'a', 'song', 'called', '&quot;', 'Nothing', 'To', '<unk>', '.', '&quot;', '<eos>'], ['When', 'I', 'was', 'little', ',', 'I', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'I', 'grew', 'up', 'singing', 'a', 'song', 'called', '&quot;', 'Nothing', 'To', 'Envy', '.', '&quot;'], ['Khi', 'tôi', 'còn', 'nhỏ', ',', 'Tôi', 'nghĩ', 'rằng', 'BắcTriều', 'Tiên', 'là', 'đất', 'nước', 'tốt', 'nhất', 'trên', 'thế', 'giới', 'và', 'tôi', 'thường', 'hát', 'bài', '&quot;', 'Chúng', 'ta', 'chẳng', 'có', 'gì', 'phải', 'ghen', 'tị', '.', '&quot;'])
(['<cls>', 'And', 'I', 'was', 'very', 'proud', '.', '<eos>

In [8]:
def make_batch(data, batchsize):
  bb = []
  ben = []
  bvi = []
  for en,vi in data:
    ben.append(en)
    bvi.append(vi)
    if len(ben) >= batchsize:
      bb.append((ben, bvi))
      ben = []
      bvi = []
  if len(ben) > 0:
    bb.append((ben, bvi))
  return bb

train_data = make_batch(train_data, BATCHSIZE)

for i in range(2):
  print(train_data[i])

([['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>']], [['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>']])
([['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>']], [['<cls>', '<eos>'], ['<cls>', '

In [9]:
def padding_batch(b):
  maxlen = max([len(x) for x in b])
  for tkl in b:
    for i in range(maxlen - len(tkl)):
      tkl.append('<pad>')

def padding(bb):
  for ben, bvi in bb:
    padding_batch(ben)
    padding_batch(bvi)

padding(train_data)

for i in range(3):
  print(train_data[i])


([['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>']], [['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>']])
([['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>'], ['<cls>', '<eos>']], [['<cls>', '<eos>'], ['<cls>', '

In [10]:
train_data = [([[vocabidx_en[token] for token in tokenlist] for tokenlist in ben],
               [[vocabidx_vi[token] for token in tokenlist] for tokenlist in bvi]) for ben, bvi in train_data]
test_data = [([vocabidx_en[token] for token in enprep], en, vi) for enprep, en, vi in test_data]
for i in range(3):
  print(train_data[i])
  print(test_data[i])

([[2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3]], [[2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3]])
([2, 444, 49, 96, 678, 16, 49, 884, 429, 823, 96, 22, 1203, 28, 22, 203, 16, 70, 49, 722, 218, 2403, 10, 2271, 178, 545, 9225, 868, 0, 48, 545, 3], ['When', 'I', 'was', 'little', ',', 'I', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'I', 'grew', 'up', 'singing', 'a', 'song', 'called', '&quot;', 'Nothing', 'To', 'Envy', '.', '&quot;'], ['Khi', 'tôi', 'còn', 'nhỏ', ',', 'Tôi', 'nghĩ', 'rằng', 'BắcTriều', 'Tiên', 'là', 'đất', 'nước', 'tốt', 'nhất', 'trên', 'thế', 'giới', 'và', 'tôi', 'thường', 'hát', 'bài', '&quot;', 'Chúng', 'ta', 'chẳng', 'có', 'gì', 'phải', 'ghen', 'tị', '.', '&quot;'])
([[2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3]

## Embedding

In [11]:
' '.join(train_vi[0])

'Khoa học đằng sau một tiêu đề về khí hậu'

In [12]:
vi = [a for a in train_vi]
en = [a for a in train_en]

In [13]:
import gensim
w2v_model_vi = gensim.models.word2vec.Word2Vec(train_vi,size=300, min_count=3)
w2v_model_en = gensim.models.word2vec.Word2Vec(train_en,size=300, min_count=3)

In [14]:
# Tạo ma trận Embedding
def EmbeddingMatrixCreater(word_index, w2v_model, emb_dim):
  embedding_matrix = np.zeros((len(word_index) + 1, emb_dim))
  for word, i in word_index.items():
    try:
      embedding_vector = w2v_model.wv[word]
      if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    except:
      print(f'{word} not in word2vec')
  return embedding_matrix

In [15]:
embedding_matrix_en = EmbeddingMatrixCreater(vocabidx_en, w2v_model_en, 300)
embedding_matrix_vi = EmbeddingMatrixCreater(vocabidx_vi, w2v_model_vi, 300)

<unk> not in word2vec
<pad> not in word2vec
<cls> not in word2vec
<eos> not in word2vec
<unk> not in word2vec
<pad> not in word2vec
<cls> not in word2vec
<eos> not in word2vec


In [16]:
weight_en = torch.Tensor(embedding_matrix_en)
weight_vi = torch.Tensor(embedding_matrix_vi)

In [17]:
print(weight_en.shape)

torch.Size([24421, 300])


# Transfomer

In [18]:
from torch import Tensor
from torch import nn
from torch.nn import Transformer
import math

In [19]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 900):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)
        
    # def evaluate(self):



In [20]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == vocabidx_en['<pad>']).transpose(0, 1)
    tgt_padding_mask = (tgt == vocabidx_vi['<pad>']).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [21]:
torch.manual_seed(0)

MODELNAME = 'transfomers.model'
SRC_VOCAB_SIZE = len(vocablist_en)
TGT_VOCAB_SIZE = len(vocablist_vi)
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 2

model = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM).to(DEVICE)


loss_fn = torch.nn.CrossEntropyLoss(ignore_index=1)


def train():
  optimizer = torch.optim.Adam(model.parameters(), lr=LR)
  for epoch in range(10):
    loss = step = 0

    for en, vi in train_data:
      
      en = torch.tensor(en, dtype=torch.int64).transpose(0,1).to(DEVICE)
      vi = torch.tensor(vi, dtype=torch.int64).transpose(0,1).to(DEVICE)
      tgt_input = vi[:-1, :]
      

      optimizer.zero_grad()
      src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(en, tgt_input)
      y = model(en, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)
      tgt_out = vi[1:, :]
      batchloss = loss_fn(y.reshape(-1, y.shape[-1]), tgt_out.reshape(-1))
      batchloss.backward()
      optimizer.step()
      loss = loss + batchloss.item()
      step += 1
      if step % 1000==0:
        print(f'step: {step} - loss: {loss}')
    print("epoch", epoch, ": loss", loss)
  torch.save(model.state_dict(), MODELNAME)

In [22]:
train()

step: 1000 - loss: 4911.826586961746
step: 2000 - loss: 9598.88439488411
step: 3000 - loss: 14196.313145637512
step: 4000 - loss: 18692.523679733276
step: 5000 - loss: 23138.345583200455
step: 6000 - loss: 27557.029801130295
step: 7000 - loss: 31975.408529520035
step: 8000 - loss: 36423.29042696953
epoch 0 : loss 37937.94640517235
step: 1000 - loss: 3225.7819479426835
step: 2000 - loss: 6713.591579767177
step: 3000 - loss: 10351.623601289699
step: 4000 - loss: 14067.513488861034
step: 5000 - loss: 17870.644597621867
step: 6000 - loss: 21759.174750896404
step: 7000 - loss: 25734.35247907252
step: 8000 - loss: 29813.113197179744
epoch 1 : loss 31225.176913114497
step: 1000 - loss: 2810.6737791141495
step: 2000 - loss: 5917.412997110747
step: 3000 - loss: 9205.667843922041
step: 4000 - loss: 12599.440416439436
step: 5000 - loss: 16103.995543344878
step: 6000 - loss: 19722.50711880531
step: 7000 - loss: 23451.5757449707
step: 8000 - loss: 27321.02501450386
epoch 2 : loss 28674.92731939163


In [25]:
def evaluate(model, src, max_len, start_symbol):
    src_mask = torch.zeros((src.shape[0], src.shape[0]),device=DEVICE).type(torch.bool)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    pred = []
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0)).type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        # next_word = out.squeeze().argmax()
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        # print(prob)
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == vocabidx_en['<eos>']:
            break
        pred_y = vocablist_vi[next_word][0]
        pred.append(pred_y)
    return pred

def test():
  model = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM).to(DEVICE)
  model.load_state_dict(torch.load("/content/transfomers.model"))
  model.eval()
  ref = []
  pred = []
  for enprep, en, vi in test_data:
    input = torch.tensor([enprep], dtype=torch.int64).transpose(0,1).to(DEVICE).view(-1, 1)
    p = evaluate(model, input, 50, vocabidx_en['<cls>'])
    # p = model.evaluate(model, input, maxlen,start_symbol, vocablist_vi, vocabidx_vi)
    ref.append([vi])
    pred.append(p)
  bleu = torchtext.data.metrics.bleu_score(pred, ref)
  print("total: ", len(test_data))
  print("bleu: ", bleu)

In [24]:
test()

total:  1268
bleu:  0.06139875575900078
